In [ ]:
import json

with open('./paragraphs_v1.json', 'r') as f:
    paragraphs = json.load(f)
with open('/home/xilini/par-gen/im2p_new/data/dataset_paragraphs.json', 'r') as j:
    preprocessed_paragraphs = json.load(j)
with open('/home/xilini/par-gen/im2p_new/data/new_splits.json', 'r') as i:
    splits = json.load(i)

In [ ]:
paragraphs

In [ ]:
splits

In [ ]:
curr_sentence_id = 0
from spacy.lang.en import English # updated
from autocorrect import Speller
spell = Speller(lang='en')

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated

all_images = {}
all_images['images'] = []

for par_num, paragraph in enumerate(paragraphs, 1):
    filepath = paragraph['url'].split('/')[-2]
    filename = paragraph['url'].split('/')[-1]
    imgid = par_num
    split = splits[str(filename.split('.')[0])]
    
    paragraph = paragraph['paragraph']
    doc = nlp(paragraph)
    sentences = [sent for sent in doc.sents]
    
    sent_dict = {}
    sent_dict['filepath'] = filepath
    sent_dict['filename'] = filename
    sent_dict['imgid'] = imgid
    sent_dict['split'] = split
    sent_dict['sentences'] = []
    
    sent_ids = []
    tokens_dict = {}
    
    for s in sentences:
        tokens_dict = {}
        raw_sentence = s
        #print(raw_sentence)
        tokens = [token.orth_.lower().rstrip() for token in raw_sentence if not token.is_punct]
        tokens = [t for t in tokens if t != '']
        # check mispelled
        tokens = [spell(t) for t in tokens]
        tokens_dict['tokens'] = tokens
        tokens_dict['raw'] = str(raw_sentence)
        tokens_dict['imgid'] = imgid
        tokens_dict['sentid'] = curr_sentence_id
        sent_ids.append(curr_sentence_id)
        curr_sentence_id += 1
        
        sent_dict['sentences'].append(tokens_dict)

    sent_dict['stanford_par_id'] = int(filename.split('.jpg')[0])
    sent_dict['sentids'] = sent_ids
    all_images['images'].append(sent_dict)
    #break
    #print(split)

In [ ]:
all_images['images'][-1]

In [ ]:
preprocessed_paragraphs['images'][-1]

In [ ]:
with open('/home/xilini/par-gen/im2p_new/data/dataset_paragraphs_v3.json', 'w') as w:
    json.dump(all_images, w)

In [ ]:
with open('/home/xilini/par-gen/im2p_new/data/dataset_paragraphs_v3.json', 'r') as w:
    imgs = json.load(w)

In [ ]:
max_sentence_length = 0
for elem in imgs['images']:
    for s in elem['sentences']:
        if len(s['tokens']) > max_sentence_length:
            max_sentence_length = len(s['tokens'])

In [ ]:
max_sentence_length

In [ ]:
c = 0
for i in imgs['images']:
    if i['split'] == 'train':
        c += 1
c